# NBA Games Predictor - Basic

This notebook takes all the regular season games from the NBA 2016-17 season.  Using a series of regression & supervised learning techniques, it attempts to accurately predict the outcome of basketball games.

In [1]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [2]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Configuration

In [3]:
calc_last_5_games = True
calc_win_loss = True
calc_win_loss_write_all = True
use_team_keys = False
use_team_hot_encoding = False
use_team_home_away_hot_encoding = False

## Load data from file system

In [4]:
#load games - json object
with open('../input/games.json') as gamesJsonRaw:
    d = json.load(gamesJsonRaw)

games = json_normalize(d['games'])

In [5]:
games.head(5)

,id,location,oddsBet365Away,oddsBet365Home,scoreAway,scoreHome,scoreQuarters,season,teamAwayCode,teamAwayId,teamHomeCode,teamHomeId,time
0,33941,Quicken Loans Arena,5.25,1.18,88,117,"[{'number': 1, 'scoreHome': 28, 'scoreAway': 1...",2016,NYK,83,CLE,86,2016-10-25T19:30:00.000Z
1,33942,Moda Center,3.15,1.38,104,113,"[{'number': 1, 'scoreHome': 26, 'scoreAway': 2...",2016,UTA,98,POR,97,2016-10-25T22:00:00.000Z
2,33943,Oracle Arena,4.25,1.23,129,100,"[{'number': 1, 'scoreHome': 20, 'scoreAway': 3...",2016,SAS,106,GSW,101,2016-10-25T22:30:00.000Z
3,33944,Amway Center,2.60,1.55,108,96,"[{'number': 1, 'scoreHome': 27, 'scoreAway': 2...",2016,MIA,92,ORL,95,2016-10-26T19:00:00.000Z
4,33945,Bankers Life Fieldhouse,3.20,1.37,121,130,"[{'number': 1, 'scoreHome': 33, 'scoreAway': 2...",2016,DAL,108,IND,87,2016-10-26T19:00:00.000Z


In [6]:
#load stadiums - json object
with open('../input/stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [7]:
stadiums.head(5)

,lat,lng,team
0,33.757183,-84.396278,ATL
1,42.366281,-71.062266,BOS
2,40.682650,-73.974689,BRO
3,35.224519,-80.841053,CHA
4,41.880589,-87.674149,CHI


   ## Data Preperation

### Set Valid Data Types

In [8]:
#games['season'] = games['season'].astype('float')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')
games['teamAwayId'] = games['teamAwayId'].astype('uint8')
games['teamHomeId'] = games['teamHomeId'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object

### Calculate the number of games played for season

In [9]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

# Iterate through every team
for team in games.teamAwayCode.unique():
    gameCount = 0
    homeGameCount = 0;
    
    # Iterate through each game the team is present in.
    for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].iterrows():
        gameCount += 1
        
        # Update game count for team - whether away or home.
        if game.teamAwayCode == team:
            data.loc[index]['gamesPlayedAway'] = gameCount
        else:
            data.loc[index]['gamesPlayedHome'] = gameCount

data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')
            
# Append the results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)

In [10]:
games[['teamHomeCode', 'teamAwayCode', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,gamesPlayedHome,gamesPlayedAway
45,PHI,ORL,3,4
242,MIA,MEM,16,17
1066,POR,NYK,71,72
1067,WAS,BRO,72,72
1100,BRO,PHI,74,73


## Calculate the Win/Loss % for the Home team.

In [11]:
def calc_percentage(x, y):
    return 0 if y == 0 else x / y

In [12]:
if calc_win_loss == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(
        index=range(0,len(games)), 
        columns=[
            'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
            'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
            'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
            'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
        ]
    )

    # Iterate through each game.
    for index, game in games.iterrows():

        # Home Team
        # ---------
        
        historicGames = games[(games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)
       
        homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
        homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
        awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesHome = homeGamesHome + awayGamesHome
        totalWinsHome = homeWinsHome + awayWinsHome
        
        percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
        percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
        percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)
        
        # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
        # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)
        
        # Away Team
        # ---------
        
        historicGames = games[(games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)
       
        homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
        homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])
        
        awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
        awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

        #totalGames = len(historicGames);
        #totalWins = len(
        #    historicGames[
        #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
        #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
        #    ]
        #)
        
        totalGamesAway = homeGamesAway + awayGamesAway
        totalWinsAway = homeWinsAway + awayWinsAway
        
        percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
        percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
        percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)
        
        # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
        # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)
        
        # ---------    
            
        # Update the row with the history
        data.loc[index] = [
            totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
            percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
            totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
            percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
        ]

    # Add results to the dataset
    if (calc_win_loss_write_all == True):
        games = pd.merge(games, data, left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)
    else:
        games = pd.merge(games, data[['percentageHomeWinHome', 'percentageAwayWinAway']], left_index=True, right_index=True)
        games[['teamHomeCode', 'teamAwayCode', 'percentageHomeWinHome', 'percentageAwayWinAway']].sample(5)

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Calculate the results for the last 5 games for the Home team.

In [13]:
if calc_last_5_games == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

            lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
            lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
            lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
            lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
            lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
            #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

            # Update the row with the history
            data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

    # Convert types 
    data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
    data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
    data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
    data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
    data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
    data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
    data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
    data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
    data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
    data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

### Calculate the results for the last 5 games for the Away team.

In [14]:
if calc_last_5_games == True:
    
    # Create a dataframe for the results - same size as dataset
    data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

    # Iterate through each game where the teams have played at least 5 games.
    for index, game in games.iterrows():

        # Get the last five games for the team.
        last5games = games[(games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

        if len(last5games) == 5:
            lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
            lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
            lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
            lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
            lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
            #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

            lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
            lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
            lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
            lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
            lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
            #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

            # Update the row with the history
            data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

    # Convert types         
    data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
    data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
    data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
    data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
    data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
    data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
    data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
    data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
    data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
    data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

    # Add results to the dataset
    games = pd.merge(games, data, left_index=True, right_index=True)
    
    games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

In [15]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles


337.6783477035334

## Data Cleanse
Look at data types

In [16]:
games.dtypes

id                          int64
location                   object
oddsBet365Away            float64
oddsBet365Home            float64
scoreAway                   uint8
scoreHome                   uint8
scoreQuarters              object
season                      int64
teamAwayCode               object
teamAwayId                  uint8
teamHomeCode               object
teamHomeId                  uint8
time                       object
gamesPlayedHome             uint8
gamesPlayedAway             uint8
totalGamesHome             object
totalWinsHome              object
homeGamesHome              object
homeWinsHome               object
awayGamesHome              object
awayWinsHome               object
percentageTotalWinHome     object
percentageHomeWinHome      object
percentageAwayWinHome      object
totalGamesAway             object
totalWinsAway              object
homeGamesAway              object
homeWinsAway               object
awayGamesAway              object
awayWinsAway  

Transform the Time column into a datetime object.

In [17]:
games["time"] = pd.to_datetime(games["time"])

### Convert the home and away teams to one-hot encoding 

In [18]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    # teams[['team_NYK', 'team_CLE', 'team_ATL', 'team_BOS']].head(2)
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [19]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

Drop the id fields for teams.

In [20]:
if (use_team_hot_encoding == True) | (use_team_home_away_hot_encoding == True) | (use_team_keys == False):
    games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [21]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [22]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

In [23]:
#gamesQuarters = json_normalize(data=d['games'], record_path='scoreQuarters', meta=['id'])
#gamesQuarters.head(10)

In [24]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

In [25]:
# Remove games scores ... not possible to predict the future, if you already know it :)
games = games.drop(["scoreAway", "scoreHome"], axis=1)

In [26]:
# Remove any games that have less than a 5 game history.
if calc_last_5_games == True:
    games = games[(pd.notnull(games['lastGame1WinHome'])) & (pd.notnull(games['lastGame1AtHomeAway']))]

In [27]:
# Show any NaN
games[games.isnull().T.any().T]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin


In [28]:
games.head(3)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin
0,33941,5.25,1.18,2016,1,1,0,0,0,0,...,True,True,True,True,True,True,True,True,True,True
1,33942,3.15,1.38,2016,1,1,0,0,0,0,...,True,True,True,True,True,True,True,True,True,True
2,33943,4.25,1.23,2016,1,1,0,0,0,0,...,True,True,True,True,True,True,True,True,True,False


In [29]:
msk = np.random.rand(len(games)) < 0.8
msk

train = games[msk]
test = games[~msk]

Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)

#
X_test  = test.drop(["id", "homeWin"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((987, 43), (987,), (243, 43))

In [30]:
# Reset the index
test.index = range(len(test.index))

In [31]:
X_test.sample(5)

,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,...,lastGame1WinAway,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway
593,2.20,1.71,2016,39,40,38,22,17,10,21,...,True,False,True,True,True,False,True,False,True,False
1164,2.40,1.62,2016,78,78,77,50,38,28,39,...,True,False,True,False,True,True,True,True,False,False
269,2.45,1.60,2016,18,18,17,5,8,3,9,...,True,False,True,True,True,False,True,False,True,False
528,4.00,1.26,2016,34,36,32,25,20,18,12,...,True,False,True,True,False,False,True,False,True,False
815,2.40,1.62,2016,54,54,53,22,26,11,27,...,False,True,False,False,True,False,True,True,True,True


### Logistic Regression

In [32]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

67.069999999999993

In [33]:
coeff_df = pd.DataFrame(games.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,oddsBet365Away,0.277204
36,lastGame2AtHomeAway,0.268835
42,lastGame5AtHomeAway,0.263717
38,lastGame3AtHomeAway,0.245679
31,lastGame5WinHome,0.224517
4,gamesPlayedAway,0.180447
25,lastGame2WinHome,0.140416
40,lastGame4AtHomeAway,0.110096
39,lastGame4WinAway,0.074577
10,awayWinsHome,0.060400


In [34]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_log = round((correct / total) * 100, 2)

print(correct_log, total, correct, incorrect)

61.73 243 150 93


### Support Vector Machines

In [35]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

93.010000000000005

In [36]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_svc = round((correct / total) * 100, 2)

print(correct_svc, total, correct, incorrect)

60.91 243 148 95


### k-Nearest Neighbors

In [37]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

78.620000000000005

In [38]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_knn = round((correct / total) * 100, 2)

print(correct_knn, total, correct, incorrect)

58.02 243 141 102


### Gaussian Naive Bayes

In [39]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

62.609999999999999

In [40]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_gaussian = round((correct / total) * 100, 2)

print(correct_gaussian, total, correct, incorrect)

67.9 243 165 78


### Perceptron

In [41]:
perceptron = Perceptron(max_iter = 5)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

41.840000000000003

In [42]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_perceptron = round((correct / total) * 100, 2)

print(correct_perceptron, total, correct, incorrect)

40.74 243 99 144


### Linear SVC

In [43]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

58.159999999999997

In [44]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_linear_svc = round((correct / total) * 100, 2)

print(correct_linear_svc, total, correct, incorrect)

59.26 243 144 99


### Stochastic Gradient Descent

In [45]:
sgd = SGDClassifier(max_iter = 5)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

58.159999999999997

In [46]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_sgd = round((correct / total) * 100, 2)

print(correct_sgd, total, correct, incorrect)

59.26 243 144 99


### Decision Tree

In [47]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [48]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_decision_tree = round((correct / total) * 100, 2)

print(correct_decision_tree, total, correct, incorrect)

58.85 243 143 100


### Random Forest

In [49]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [50]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_random_forest = round((correct / total) * 100, 2)

print(correct_random_forest, len(Y_pred), total, correct, incorrect)

62.96 243 243 153 90


## Model Evaluation Results

In [51]:
models = pd.DataFrame({
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Correct': [correct_svc, correct_knn, correct_log, 
              correct_random_forest, correct_gaussian, correct_perceptron, 
              correct_sgd, correct_linear_svc, correct_decision_tree],
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})
models.sort_values(by='Correct', ascending=False)

,Correct,Model,Score
4,67.90,Naive Bayes,62.61
3,62.96,Random Forest,100.00
2,61.73,Logistic Regression,67.07
0,60.91,Support Vector Machines,93.01
6,59.26,Stochastic Gradient Decent,58.16
7,59.26,Linear SVC,58.16
8,58.85,Decision Tree,100.00
1,58.02,KNN,78.62
5,40.74,Perceptron,41.84


In [52]:
result[(result['homeWin'] != result['forecast'])].sample(10)

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin,forecast
193,34992,1.50,2.70,2016,70,71,69,13,35,8,...,False,False,False,False,True,True,True,False,True,False
133,34692,2.10,1.76,2016,49,49,47,21,24,12,...,True,False,False,False,True,True,False,False,True,False
75,34334,2.80,1.45,2016,28,26,27,18,15,11,...,True,True,False,False,False,True,True,True,False,True
115,34601,1.07,9.50,2016,46,44,45,18,21,8,...,True,False,True,False,True,False,False,False,False,True
106,34566,2.05,1.80,2016,46,40,46,15,22,10,...,True,True,False,False,False,False,True,True,False,True
186,34957,2.45,1.60,2016,68,69,67,37,34,19,...,True,False,True,False,False,False,False,False,False,True
221,35099,2.70,1.50,2016,77,78,76,43,39,27,...,False,False,False,True,False,True,False,False,True,False
49,34230,2.25,1.68,2016,19,20,18,7,9,3,...,True,True,False,True,False,True,False,False,False,True
63,34276,1.64,2.35,2016,22,23,20,11,8,5,...,False,True,False,True,False,True,True,False,True,False
152,34775,3.00,1.41,2016,55,56,53,37,28,23,...,False,False,False,False,True,True,True,False,True,False


In [56]:
result[(result['id'] == 33946) | (result['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin,forecast
115,34601,1.07,9.5,2016,46,44,45,18,21,8,...,True,False,True,False,True,False,False,False,False,True


In [55]:
games[(games['id'] == 33946) | (games['id'] == 34601)]

,id,oddsBet365Away,oddsBet365Home,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,...,lastGame1AtHomeAway,lastGame2WinAway,lastGame2AtHomeAway,lastGame3WinAway,lastGame3AtHomeAway,lastGame4WinAway,lastGame4AtHomeAway,lastGame5WinAway,lastGame5AtHomeAway,homeWin
5,33946,10.50,1.06,2016,1,1,0,0,0,0,...,True,True,True,True,True,True,True,True,True,True
657,34601,1.07,9.50,2016,46,44,45,18,21,8,...,True,True,False,True,False,True,False,False,False,False
